# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [6]:
browser = webdriver.Chrome(executable_path='./chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
# ref: https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-in-dynamically-loading-webpage/48851166#48851166
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 「雙11大爆塞」網站當機！流量破表...東森購物延長活動至11/12

記者洪菱鞠／台北報導
▲▼王品餐券跟iPhone手機都是大家搶購的項目。（圖／翻攝自ET Mall）
刷破百萬元的消費者，下單商品以3C與餐券為主，包括iPhone、iPad、Apple Watch等Apple產品，以及Vivo V17、SONY Xperia、OPPO A3，餐券則是王品集團、饗食天堂、漢來。進一步觀察整體消費群族輪廓，女性65%、男性35%，其中以北部會員佔大宗，40～49歲為消費主力。
▲民生必需品也十分搶手。（圖／翻攝自ET Mall）
各項商品銷售件數表現上，舒潔衛生紙榮登第1名銷售寶座，其次是王品西堤餐券，第3名為iPhone 11 128G，第4～10名依序是：王品夏慕尼餐券、AirPods Pro、春風衛生紙、王品陶板屋餐券、王品西堤餐券、王品聚餐券。銷售總額前20大排行依序：iPhone系列手機、王品集團餐券、AirPods Pro、自然美賦活膠囊組、iPad、SONY Xperia、Apple Watch，總銷金額超過8,000萬元。
▲東森購物11/11一日限定全天LINE POINTS 5%回饋無上限。
▲7-ELEVEN超取服務正式在東森購物上線。（圖／東森購物提供）
----------------------------------------------------------------------

[1] 前10月稅收達2兆64億元　估今年短徵近千億元

▲財政部。（圖／資料照）
記者吳靜君／台北報導
財政部今（11）日公布稅收統計，10月實徵淨額為1712億元，較去年同期減少754億元，年減30.6%；累計前十月實收淨額為2兆64億元，較去年同期減少1095億元，年減5.2%。
10月實徵淨額與去年同期比較，營利事業所得稅減少750億元減少最多；但是證交稅與土增稅各增12億元。
累計前10月營所稅實徵淨額營較去年同期減少1709億元，貨物稅減少74億元、房屋稅減少23億元最多；而證交稅10月的實徵淨額為99億元較去年同期增加12億元，年增率14.1%，累計前十月累計實徵淨額1169億元，較去

▲換手機、換門號又怕所存發票變不見。（圖／記者許力方攝）
記者吳靜君／台北報導
「雙11」到來，許多人買爆，尤其是最新款的手機又引誘許多民眾想要購入新手機，不過有網友常問「如果換手機號碼，手機條碼會失效嗎？原本存入的發票還會在嗎？會不會不見」。
網友的心聲財政部小編聽見了，今（11）日「雙11」發好康文，親授兩招讓民眾想換手機、想換門號等都不用擔心原本的儲存的發票會不見、繼續開開心心兌獎。
財政部小編先解釋，手機條碼是由一組手機號碼搭配E-mail申請而成，所以手機號碼及驗證碼（密碼）登入App即可顯示，之後就算換手機號碼也不會影響已經產出的手機條碼。所以民眾換門號或者換手機重新下載App有兩招可解。
一，沿用原本的手機條碼：登入舊手機號碼加驗證碼，適合記性好的人。
二，再申請新的手機條碼：用新手機號碼申請新的手機條碼，再上財政部電子發票整合服務平台把原本的手機條碼歸戶到新手機條碼。
財政部小編說，「當然一切前提是得要先有手機可以換」。
▼換門號發票不會不見的妙招。（圖／翻攝財政部臉書）
----------------------------------------------------------------------

[11] 鬆綁！欠稅擔保品　新增上櫃股票、不動產

▲財政部台北國稅局。（圖／記者湯興漢攝）
記者吳靜君／台北報導
納稅義務人如果無法一次繳清稅捐，可以就欠繳應納稅額提供擔保，就不會立即被強制執行。為了使納稅人提供作為繳納稅款擔保品的範圍及計算價值更為明確，財政部修正，將黃金改以「賣出價」計算、也放寬上櫃的有價證券、不動產作為擔保品。
財政部發布「黃金外幣及上市有價證券作為擔保品計值辦法」名稱修正為「稅捐稽徵機關受理擔保品計值及認定辦法」。
過去只有上市公司有價證券作擔保品，但是實務上國稅局會也有納稅人也有拿上櫃股票來擔保，國稅局則會依照個案認定，修正之後就可以將上櫃股票納入擔保當中；但是興櫃的股票則因為市場較小、漲跌波動度大，因此沒有列入，但是國稅局實務上也仍可依個案認定。
新增上櫃公司的有價證券作為擔保品，如果是股票或者受益憑證，就比照上市的有價證券原則按照提供擔保品日前一日的證券交易的收盤價八成；如果是公司債則按照面額八成計算價值。
至於不動產的部分，過去是規範在解釋令當中，納稅人可提供「易於變價、無產權糾紛且能足額清償的土地或已辦妥

▲尖端醫董事長蘇文龍（右）與銘福集團總裁陳福松（左）簽署MOU。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
尖端醫（4186）今（11）日與銘福集團旗下天福天美仕簽署合作備忘錄(MOU)，雙方將進行細胞領域整廠輸出、細胞治療與細胞儲存等相關事務合作，搶攻以骨髓間質幹細胞治療退化性關節炎、免疫細胞治療實體腫瘤癌等商機，並預計2022年導入漳浦天福醫院、養生村。
尖端醫董事長蘇文龍表示，尖端醫將提供細胞領域整廠輸出，協助協助幹細胞實驗室的軟硬體建置、人員教育、國際認證等，並導入尖端臍帶血、期待經驗的細胞儲存業務，而天福天美仕針對東南亞及中國大陸天福醫院及其他潛在合適單位進行相關細胞技術洽談與推廣。
▲尖端醫董事長蘇文龍。（圖／記者姚惠茹攝）
蘇文龍指出，尖端醫通過的特管辦法核准的自體細胞治療，包括自體免疫細胞(CIK)治療一至四期實體癌，並已收案10多名病患，預計第4季可望貢獻營收，而目前合作送申請案的醫療院所，還有包括花蓮慈濟醫院、新光醫院、臺北醫學大學附設醫院及萬芳醫院等。
蘇文龍分享，尖端醫隨後又獲准收案治療退化性關節炎、膝關節軟骨缺損，成為全國第一個符合人體細胞組織優良操作規範（GTP）骨髓間質幹細胞製備場所，第三次通過細胞治療技術案件，並即將展開收案，更是台灣少數同時布局在幹細胞和免疫細胞兩個領域的細胞應用公司。
▲銘福集團總裁陳福松。（圖／記者姚惠茹攝）
銘福集團總裁陳福松表示，銘福集團與天福集團長期經營中國大陸市場，目前在大陸有天福茶業、漳州有科技學院、漳浦有天福醫院及養生村，因此期望透過旗下天福天美仕與尖端醫合作，導入細胞領域整廠輸出到海外。
陳福松指出，福建漳浦天福醫院為中國三級綜合醫院，預計2022年完工使用，並將引進台灣先進的醫療養護管理模式，因此雙方合作將會以天福醫院為基礎，提供當地病患更多境內外醫療與行銷資源，串接更多細胞技術的合作類型。
▲尖端醫攜手天福天美仕簽署MOU。（圖／記者姚惠茹攝）
----------------------------------------------------------------------

[22] 明年股市樂觀　法人：科技股有助台股延續多頭表現

記者陳心怡／台北報導
不過，凱基投顧提到，股市短線必須注意在選舉爭議下，風險偏好將下降，屆時可能會看到評價面驅動的股市下修，而美元短線也將因避險需求而回

▲蘋果今凌晨推出3款新Mac。（圖／Apple直播）
記者林妤柔／綜合報導
蘋果今（11）日發表3款新Mac，以及首款為Mac而設的Apple Silicon 處理器「M1」，正式揮別Intel X86 平台。這3款新品的性能大幅升級，但FaceTime鏡頭的硬體解析度只有720p。
由於新冠肺炎疫情大流行，全世界盛行遠距上班，MacBook的720p FaceTime相機將提供更出色的體驗，從Mac內建軟體到App好比Reincubate Camo，能將iPhone鏡頭當作Mac視訊鏡頭。
另外，蘋果還靠著新圖像訊號處理器（ISP）改善鏡頭，包括降低噪音、增加大動態範圍、改善自動白平衡和增強ML人臉探測技術，可惜只有720p解析度。然而，是否真改善用戶體驗，仍要等待時間證明。
這3款筆電分別為MacBook Air，售價只要3萬900元起，教育優惠價2萬7800元起；13 吋 MacBook Pro售價為3萬9900元起，教育優惠價3萬6800元起；Mac mini售價2萬1900元起。
雖然蘋果在發表會中提到，在今天已經可以下訂，下周就可以開始陸續到貨，不過，從蘋果台灣官網上可以看到，台灣並未在首波販售名單之中，因此台灣果粉如果想購買的話，還需要再等等。
----------------------------------------------------------------------

[33] iPhone摺疊機傳在2022年秋季登場！　將取代 iPad mini

▲iPhone摺疊機傳在2022年秋季登場。（圖／截自ConceptsiPhone）
記者謝仁傑／綜合報導
有關可折疊 iPhone 的傳聞已經有一段時間了，甚至傳聞蘋果正在用三星的摺疊機顯示器做來樣本測試，現有外媒報導，可摺疊 iPhone 將於 2022 年秋季登場。
正如先前所曝光的各種專利圖所示，蘋果勢必遲早會發布一款可摺疊設備。預計 Apple 首款可摺疊 iPhone 會具備 8GB RAM 和至少 256GB 儲存量，基本型號的價格約 1,499 美元（約新台幣 42,720 元），將會在 2022 年 11 月登場。
另一方面，Google 顯然正在加緊發布 Pixel 摺疊機的準備，LG 和微軟目前也已發表了雙螢幕手機，甚至有報導指出，未來摺疊手機將會成為主流。
---

▲螞蟻金服最大規模IPO突然卡關，市場震驚。（圖／視覺中國）
記者吳靜君／台北報導
行政院前院長、新世代金融基金會董事長陳冲今（11）日「雙11」之日發表《螞蟻不上市 見微早知著》一文，討論螞蟻金服上市卡關，以及金融科技（Fintech）與金融監理之間的平衡問題。
11月3日美國大選，硝煙四起；而太平洋的另岸的金融市場，高達345億美元最大IPO，遭上海交易所、香港交易所同日發出暫緩上市決定，在11月5日將掛牌之際，其衝擊情況可以想見。
外界初步反應，多認為此事與阿里巴巴集團創辦人馬雲在10月24日在外灘金融峰會演講時言詞犀利，當時馬雲也說出「好的方式不怕監管，但怕用昨天的方式去管」；聽起來有點像陳冲所屬的新世代金融基金會近年所主張「數位時代不能用石器時代的法規」的觀點。
馬雲或許是急於為螞蟻金服的生存及未來的發展舖路喊話，以致於衝撞中國國家副主席王岐山「金融創新要防範系統性金融風險」的論調。但是如果觀察10月底中國國務院金融發展委員會的結論，以及11月1日及2日連續公開徵求意見的金融控股公司監督管理辦法、網路小額貸款管理暫行辦法，加上11月2日四大金融監理機關的約談，層層相扣應該不是即興之作，甚至是亡羊補牢的策略。
▲行政院前院長陳冲。（圖／記者陳政錄攝）
陳冲提到，近年互聯網支付業者在大陸蓬勃發展，該項業務金額龐大，如果沒有妥善的法規日久將會尾大不掉。他認為，單純支付服務是一回事、利用帳上餘額投資、借貸又是另一回事，單純支付已經是頭痛的問題，再加上螞蟻金服的大張旗鼓自然更引人側目。
陳冲指出，他曾說處理金融問題台灣是先緊後鬆，但是大陸是先鬆後緊，也逐步應驗。大陸的支付業者開始感受壓力，大陸的第三方支付機構統一清算平台2017年7月3日開始試營運、2018年6月30日正式收編所有網路金融進行更大的管控。目的就是要支付寶、微信等支付業者手中的大數據資料繳交由中央統一控管，終結了大陸金融大數據業者的壟斷。
依陳冲之見，螞蟻金服未能上市並非一朝一夕之事，以往互聯網業者一向將螞蟻金服定位為FinTech，是互聯網的金融平台透過例如支付寶的「花唄借唄信貸業務」（目前已超過人民幣2兆元）掌握龐大數據、涉及廣大群眾權益，但尚未納入金融監理。陳沖指出，Fintech與金融業者中間，仍有一條fine line，金融監理機關必須重新審視金融法規，用數位時代的眼光，檢討社會未來的需要。

▲麗彤生醫董事長張麗綺。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
麗彤生醫（6539）公布10月合併營收1.7億元，月增11.82％，年增8.96％，主要受惠於醫美產業旺季到來，連續三個月單月營收年增率正成長，並刷新今年單月營收新高；累計今年前10月合併營收達14.65億元，整體營運相當穩健。
麗彤表示，10月營收成長主要受惠旗下DV保健食品、DV保養品、DV皮膚管理中心與醫材等皆有良好的銷售表現，以營收佔比來看，醫材銷售在第4季醫美產業旺季期間有強勁的銷售業績，帶動包含含DV皮膚管理中心的醫美管理平台營運規模持續放大。
麗彤指出，旗下自有品牌的DV保健食品自5月起陸續將DV醇耀妍、雙樂纖兩大新品加入銷售行列後，並在線上、線下各大銷售通路備貨力道加大下，營收比重站穩30％水準，雙引擎動能火力全開，推升10月營收飆出1.65億元以上，刷新今年單月營收新高紀錄。
麗彤進一步指出，因應「新零售」概念崛起，公司自今年6月更名為「麗彤生醫」後，重新定位公司的競爭優勢，更強化完善多元銷售通路的布局，以平台整合能力為核心基礎，透過新產品、新通路、新會員提高市場佔有率。
麗彤強調，線上通路主打「高性價比」的DV保健食品、保養品，滿足消費者追求由內而外自然散發健康美麗的訴求，同時藉由整合線上DV官網18萬名、線下DV皮膚管理中心以及策略聯盟醫美診所的34萬名會員的整體消費動能，並為迎接雙11購物旺季到來，可望挹注整體營運維持良好成長動能。
展望今年第4季，麗彤表示，樂觀看好醫美保健消費旺季，並在雙十一電商購物潮、保健及保養品再度推出優惠產品組合，以推升營收表現，而策略聯盟醫美診所也祭出1111相關優惠活動助威，催動主力醫材產品包括肉毒、玻尿酸、皮秒雷射及超音波拉皮設備，搶佔更多醫美市場份額。
----------------------------------------------------------------------

[54] 兩岸經濟脫鉤？　國台辦：不是某些人和勢力能破壞得了

▲大陸國台辦新聞局副局長兼發言人朱鳳蓮。（圖／記者魏有德攝）
記者任以芳／北京報導
兩岸經濟可能脫鈎嗎？根據統計數據顯示，兩岸貿易呈現逆勢增長趨勢。對此，大陸國台辦發言人朱鳳蓮今（11日）表示，「兩岸經貿合作符合市場經濟規律，具有強大的抗干擾能力，不是某些人某些勢力想脫鈎、想阻撓就能破壞得

▲謝金河表示台股將進入大時代。（圖／記者湯興漢攝）
記者林妤柔／綜合報導
財信傳媒董事長謝金河今（11）天在臉書發文指出「台股將進入大時代」，自1990年台股寫下12682的歷史新紀錄後，從此進入蟄伏的卅年。
謝金河指出，國泰投信有兩張圖表十分珍貴，一張是台股指數和企業盈利的趨勢圖，另一張是美國股市，從盈餘成長的角度來看，台股未來會比美股更強勁。
謝金河將今年全體上市櫃公司營收及每季淨利整理出一張表格，指出今年疫情蔓延全球，台灣的上市櫃公司營收從七月起站上3兆元大關，今天公告10月營收3.24兆元，月增1.8%，年增0.8%，也是今年最高的營收紀錄。
往後看，台灣的出口連續三個月超過300億美元，10月達322.3億美元，年增11.2%，以半導體為主的電子零組件出口達到129.7億美元，年增21.8%，可說十分強勁。10月營收公告後，謝金河預估第三季淨利將超過6000億以上，這兩天謎底就會揭曉。
他表示，今年第一季全體上市櫃公司獲利3510億元，第二季5412億元，第三季淨利更進一步提升，台積電一季賺1371億，聯電91億，都是新高紀錄，最難得的好，過去慘賠的面板集團到第三季轉虧為盈，太陽能第二季開始賺錢，這些原本吃掉利潤的慘業，全部出現獲利，第三季浄和平有機會創下歷史新紀錄，這是基本面支持台股向上的後盾。
台股將進入大時代 1990年台股寫下12682的歷史新紀錄後，從此進入蟄伏的卅年，前不久，國泰投信董事長張錫說，他進入資本市場從來沒有見過如此美好的時光，他認為台股有四光環，明年將飆更高。這四大光環指的是，殖利率高，高科技起飛，半導體大缺貨...
----------------------------------------------------------------------

[63] 台美經濟繁榮夥伴對話20日登場　聚焦5G網絡、供應鏈重組議題

▲在美國國務卿蓬佩奧（左2）推薦下，柯拉克（右2）2019年1月獲得總統川普提名為國務次卿。（圖／達志影像／美聯社）
記者林淑慧／台北報導
美國國務卿蓬佩奧證實，首屆「臺美經濟繁榮夥伴對話」將於11月20日在華府及台北兩地以實體結合線上方式舉行，經濟部長王美花今(11)日表示，雙方將聚焦5G乾淨網絡、供應鏈、投資審查、基礎建設及能源等議題進行討論。
國務院次卿柯拉克 20 日將於華府主持美國與台灣的經濟對

記者謝仁傑／綜合報導
蘋果今（11）日的發表會中，將重點放在自家新開發的「M1」晶片，號稱是迄今為止打造的最強大晶片，也是專為 Mac 設計的第一款晶片。M1 針對 Mac 系統最佳化，因此小巧的尺寸與能源效率至關重要。
M1 為系統單晶片（SoC），能在單一晶片上結合多項強悍技術，更具備統一記憶體架構，可大幅提升效能與效率表現，且它是第一個使用先進 5 奈米製程技術所打造的個人電腦晶片，滿載 160 億個電晶體，是蘋果迄今裝載在晶片上最多的數量。
在低功耗晶片中，M1 配備全球最快速的 CPU 核心、全球最強的每瓦 CPU 效能、全球個人電腦中最快速的整合式繪圖處理 ，以及帶來非凡機器學習效能的 Apple 神經網路引擎，因此可提供最快達 3.5 倍的 CPU 效能、最快達 6 倍的 GPU 效能以及最快達 15 倍的機器學習速度，還有較以往 Mac 長達 2 倍的續航力。
Apple 硬體技術資深副總裁表示，「M1 是為 Mac 打造的突破性 SoC，沒有其他晶片能比擬。十多年來，我們致力於設計領先業界的晶片，並用於 iPhone、iPad 與 Apple Watch 上，M1 奠基於這個基礎，並且帶領 Mac 邁向新時代。在低功耗晶片中，M1 擁有全球最快速的 CPU 核心、全球個人電腦中最快速的整合式繪圖處理，以及帶來非凡機器學習效能的 Apple 神經網路引擎。具備精彩的效能、強大的功能以及出色的效率等獨特優勢，M1 是我們歷來所打造的最佳晶片。」
過去 Mac 與 PC 依賴多個晶片來處理 CPU、I/O、安全等眾多功能，M1 將這些技術結合在單一 SoC 上，整合更非凡效能與能源效率的方式提升至全新境界。M1 也具備統一記憶體架構，能在自訂的封裝中整合高頻寬、低延遲的單一記憶體池，可讓 SoC 中的所有技術能夠存取同樣的資料，不用來回複製其他記憶體池中的內容，進一步改善效能與效率。
M1 配備 8 核心 CPU，其中包括 4 個高效能核心和 4 個高效率核心，針對單執行緒工作，每個高效能核心都能以最節能的方式提供領先業界的效能。在低功耗晶片領域中，這些核心是全球最快速的 CPU 核心，可讓攝影師以飛快速度編輯高解析度照片，並且讓開發者打造 app 的速度比以往幾乎快上 3 倍，而 4 個高效能核心可同時運作，大幅提高執行多執行緒任務的效能表現。
4 個

▲聯發科發布第三款5G新晶片天璣700。（圖／聯發科提供）
記者周康玉／台北報導
聯發科今（11）日凌晨發布第三款5G智慧手機晶片——天璣700，採用7奈米製程，主打大眾市場，持續擴增在5G系統單晶片（SoC）產品實力及廣度。
隨著5G通訊的起飛，聯發科天璣系列提供全面覆蓋旗艦、高端、中端和大眾市場的多樣選擇 ，充分滿足市場需求。
聯發科副總經理暨無線通訊事業部總經理徐敬全表示：「隨著天璣系列產品組合的不斷擴展，我們將最新的5G功能帶到各層級的手機市場，讓更多用戶可以享受5G高速體驗。天璣700支援先進的5G連接、夜拍增強等拍攝功能和全球多種語音助理，並採用高能效的整合式設計，以先進的技術及優異規格朝5G普及化邁進。」
天璣700採用八核心CPU架構，包括兩顆大核Arm Cortex-A76，主頻高達2.2GHz。天璣700支援先進的5G技術，包括5G雙載波聚合（2CC 5G-CA）和5G雙卡雙待（DSDS），以及更高速且清晰的5G VoNR(Voice over new radio )語音服務。
聯發科去年推出旗下首款5G SoC晶片天璣1000之後，主打旗艦、高階市場，第一季開始放量；今年初再推出天璣800，主打中階市場，第二季開始出貨；今日推出第三款手機單晶片--天璣700，主打大眾市場。
天璣700主要功能和規格包括：
----------------------------------------------------------------------

[81] 聯發科推2款量身打造Chromebook晶片組　最快明年Q2上市

▲聯發科推出最新為Chromebook設計的晶片組MT8195和 MT8192 。（圖／聯發科提供）
記者周康玉／台北報導
MT8195還整合聯發科獨家AI處理器--APU3.0，可提供每秒平均運算效能高達4 TOPs的性能，強大的五核Arm-Mali G57 GPU強化視覺，四通道2133MHz LPDDR4X確保強大的內核擁有充足的記憶體頻寬；還支援多達三台顯示幕同時顯示。
聯發科MT8192晶片組整合8核CPU，包括4個Arm Cortex-A76和4個Arm Cortex-A55；APU 2.0為MT8192提供了高達2.4 TOPs的最高性能，還搭載5核Arm-G57 GPU。此外還擁有2133MHz的LPDDR4

▲開發金（圖／資料照）
記者吳靜君／台北報導
開發金今（10）日公布獲利，10月稅後純益為11.86億元，較9月的9.4億元略為成長；今年前10月累計稅後純益為99.77億元，每股稅後純益（EPS）為0.68元。
開發金發言人張立荃表示，受到美國大選影響及歐美傳出疫情升溫，10月資本市場在高檔盤整，成交量同步縮減，子公司凱基銀與凱基證仍持續展現穩定獲利動能。
凱基銀行10月份獲利主要來自利息及手續費收入等核心收益及金融資產投資獲利貢獻，合計稅後獲利4.32億元，累計前十月稅後獲利33.45億元。凱基證券各項業務表現穩健，合計稅後獲利9.16億元，累計前十月稅後獲利65.06億元。
中華開發資本因投資部位評價表現受到市場波動而漲跌互見，十月份稅後虧損1.50億元，累計前十月稅後虧損11.04億元。中國人壽10月份單月獲利8.05億元；累計前十月獲利156.45億元。
----------------------------------------------------------------------

[93] 量販超市雙11優惠懶人包！「搶5折家電、抽萬元禮券」線上線下開打

▲全聯「雙11購物節」限時3天線上線下同步優惠，實體門市超過300項商品買1送1起。（圖／全聯提供）
記者林育綾／綜合報導
迎戰「雙11購物節」，各大通路紛紛祭出超殺優惠，今年多家量販超市也皆結合線下及線上資源，端出電商及實體門市優惠，如愛買線上購物推出限時「iPhone 12享73折」；全聯電商平台也首賣Switch限時限量91折、狂送1111點數，門市也推出多項買1送1；家樂福線上購物消費滿額1111元享11%折扣，還舉辦直播「搶5折家電、抽萬元禮券」。《ETtoday新聞雲》為您整理懶人包，想趁機省一波快筆記！
【家樂福】
家樂福線上購物在先前搶先推出「雙11暖身」至11/10（含），凡購買線上購物活動商品（依商品頁標示折扣為主），單筆滿 1111元現折7%，再送1111元線上折價劵。而活動期間訂單皆可登記前1500名即「送111元購物金」。而實體通路則有330件熱門人氣商品「買一送一」；另有多個品牌「好禮大放送」，包括茶茶小王子指定商品滿1555元，即贈台灣米棒1包等。
▲家樂福雙11直播「搶5折家電、抽萬元禮券」。（圖／業者提供）
另外家樂福線上購物全站自11/11至11/13